In [1]:
%reload_ext autoreload
%autoreload 2

In [3]:


from alpharaw.mzml import MzMLReader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
from alphabase.spectral_library.base import SpecLibBase

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [4]:
base_folder = '/Users/klaproth-andrade/Documents/alpha_hack' #"/Users/georgwallmann/Documents/data/alphadia-validate/"
data_folder = f'{base_folder}/data'
output_folder = f'{base_folder}/output'

#### Obtain raw & results data
Note: this is just required if you did not run the search yourself in search_1.10.0.ipynb

In [6]:
for folder in [data_folder, output_folder]:
    if not os.path.exists(folder):
        os.makedirs(folder)

# Bulk injections of HeLa cell lysate acquired on the Orbitrap Astral
raw_data_url_list = [
    #"https://datashare.biochem.mpg.de/s/339jg5HtGrwLwDN/download?files=20231017_OA2_TiHe_ADIAMA_HeLa_200ng_Evo011_21min_F-40_07.raw",
"https://datashare.biochem.mpg.de/s/Nsp8CaHMBf7FHq1/download?path=%2Fraw_data&files=20231023_OA3_TiHe_ADIAMA_HeLa_200ng_Evo01_21min_F-40_iO_after_01.raw",

]

# results from search_1.10.0.ipynb
precursors_tsv_url = "https://datashare.biochem.mpg.de/s/fsCqlT757ttVWI8"
speclib_url = "https://datashare.biochem.mpg.de/s/VxakpS6mhM2IwxJ"


from alphadia.test_data_downloader import DataShareDownloader
raw_file_paths = [DataShareDownloader(url, data_folder).download() for url in raw_data_url_list]
precursors_tsv_path = DataShareDownloader(precursors_tsv_url, output_folder).download()
speclib_path = DataShareDownloader(speclib_url, output_folder).download()

/Users/klaproth-andrade/Documents/alpha_hack/data/20231023_OA3_TiHe_ADIAMA_HeLa_200ng_Evo01_21min_F-40_iO_after_01.raw already exists
/Users/klaproth-andrade/Documents/alpha_hack/output/precursors.tsv already exists
/Users/klaproth-andrade/Documents/alpha_hack/output/speclib.hdf already exists


### 1. Set up data
First we will point our notebook to the raw file and search results obtained using this raw file.

We will then load three objects into the notebook:
- The raw DIA data `dia_data`
- The search results `precursor_df`
- The spectral library `spectral_library`

In [8]:
current_raw_name = os.path.basename(raw_file_paths[0]).replace('.raw', '')
current_raw_path = raw_file_paths[0]

In [9]:
precursor_df = pd.read_csv(precursors_tsv_path, sep='\t')
precursor_df = precursor_df[precursor_df['run'] == current_raw_name]

spectral_library = SpecLibBase()
spectral_library.load_hdf(speclib_path)

In [10]:
from alphadia.data.alpharaw_wrapper import Thermo
dia_data =  Thermo(current_raw_path)

49it [00:25,  1.96it/s]


### 2. Inspect data
Next we want to inspect the data structures to get a better understanding of the data.

#### 2.1 Spectral library
We will start with the spectral library.
It's an alphabase SpecLibBase object, which contains the following attributes:
- `precursor_df`: a pandas DataFrame containing the precursor information
- `fragment_mz_df`: a pandas DataFrame containing the fragment m/z values
- `fragment_intensity_df`: a pandas DataFrame containing the fragment intensities

The `precursor_df` links to the fragment_mz_df and fragment_intensity_df via the `frag_start_idx` and `frag_stop_idx` columns.
For unique indexing we will use the `mod_seq_charge_hash` column.

In [14]:
#set(spectral_library.precursor_df.mods)

In [15]:
spectral_library.precursor_df[['precursor_mz', 'sequence', 'mods', 'mod_sites', 
                               'charge', 'mod_seq_charge_hash', 'frag_start_idx', 'frag_stop_idx']].head()

,precursor_mz,sequence,mods,mod_sites,charge,mod_seq_charge_hash,frag_start_idx,frag_stop_idx
0,474.764789,MIIWLEK,Oxidation@M,1,2,6778347273406786435,0,6
1,466.220746,FYSQQMK,,,2,15522010155980186642,6,12
2,409.218726,ADIEEIK,,,2,1205537395403030235,12,18
3,817.430175,ADIEEIK,,,1,1205537395403030234,18,24
4,451.215463,MFVSDFR,,,2,11088622922221221635,24,30


In [16]:
spectral_library.precursor_df

,cardinality,channel,charge,decoy,elution_group_idx,frag_start_idx,frag_stop_idx,genes,i_0,i_1,...,mods,mono_isotope_idx,nAA,nce,precursor_idx,precursor_mz,proteins,rt_norm_pred,rt_pred,sequence
0,1,0,2,0,0,0,6,TM245_HUMAN,0.553370,0.298520,...,Oxidation@M,0,7,25,7188,474.764789,Q9H330,0.576842,0.576842,MIIWLEK
1,1,0,2,0,1,6,12,SOS1_HUMAN,0.568416,0.289767,...,,0,7,25,3685,466.220746,Q07889,0.174658,0.174658,FYSQQMK
2,1,0,2,0,2,12,18,NUCB2_HUMAN,0.643259,0.269783,...,,0,7,30,376694,409.218726,P80303,0.184828,0.184828,ADIEEIK
3,1,0,1,0,3,18,24,NUCB2_HUMAN,0.643259,0.269783,...,,0,7,25,3443,817.430175,P80303,0.184828,0.184828,ADIEEIK
4,1,0,2,0,4,24,30,CDC45_HUMAN,0.575737,0.286920,...,,0,7,30,479958,451.215463,O75419,0.504794,0.504794,MFVSDFR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256662,1,0,5,0,256629,3418857,3418908,YBOX3_HUMAN,0.117547,0.272365,...,,0,52,25,235307,902.247502,P16989,0.612353,0.612353,SPVGSGAPQAAAPAPAAHVAGNPGGDAAPAATGTAAAASLATAAGS...
256663,1,0,5,0,256630,3418908,3418959,ELOA1_HUMAN,0.080857,0.228995,...,,0,52,25,235309,1066.910275,Q14241,0.716985,0.716985,IKPAPYPMGSSHASASSISFNPSPEEPAYDGPSTSSAHLAPVVSST...
256664,1,0,4,0,256631,3418959,3419010,YBOX3_HUMAN,0.117547,0.272365,...,,0,52,25,235308,1127.557558,P16989,0.612353,0.612353,SPVGSGAPQAAAPAPAAHVAGNPGGDAAPAATGTAAAASLATAAGS...
256665,1,0,5,0,256632,3419010,3419061,KDM1A_HUMAN,0.125460,0.279277,...,,0,52,25,235305,881.228655,O60341,0.676795,0.676795,AAAAAAAAAAAATGTEAGPGTAGGSENGSEVAAQPAGLSGPAEVGP...


In [17]:
spectral_library.fragment_mz_df.head()

,b_z1,b_z2,y_z1,y_z2
0,148.042679,74.524979,801.486877,401.247101
1,261.126740,131.067001,688.402832,344.705048
2,374.210815,187.609039,575.318787,288.163025
3,560.290100,280.648682,389.239471,195.123367
4,673.374207,337.190735,276.155396,138.581329


In [18]:
spectral_library.fragment_intensity_df.head()

,b_z1,b_z2,y_z1,y_z2
0,0.000000,0.0,0.125327,0.000000
1,0.335304,0.0,1.000000,0.049048
2,0.282448,0.0,0.614709,0.001847
3,0.058418,0.0,0.219854,0.000000
4,0.024189,0.0,0.172246,0.000000


In [22]:
from alphabase.spectral_library.flat import SpecLibFlat
spectral_library_flat = SpecLibFlat()
spectral_library_flat.parse_base_library(spectral_library)
spectral_library_flat.fragment_df

,mz,intensity,type,loss_type,charge,number,position
0,801.486877,0.125327,121,0,1,6,0
1,261.126740,0.335304,98,0,1,2,1
2,688.402832,1.000000,121,0,1,5,1
3,344.705048,0.049048,121,0,2,5,1
4,374.210815,0.282448,98,0,1,3,2
...,...,...,...,...,...,...,...
5465187,630.320557,0.124205,121,0,1,6,44
5465188,573.299072,0.029776,121,0,1,5,45
5465189,444.256500,0.078959,121,0,1,4,46
5465190,343.208832,0.125994,121,0,1,3,47


| Column | Description |
|--------|-------------|
| mz | Mass to charge ratio of the fragment ion |
| intensity | Relative intensity of the fragment ion |
| type | Type of fragment series. Use `chr(type)` to get type:<br>`a = 97, b = 98, c = 99, x = 120, y = 121, z = 122` |
| loss_type | Fragment loss type:<br>`0 = none, 18 = H2O, 17 = NH3` |
| charge | Charge of the fragment ion |
| number | Fragment series number in direction of series (one indexed).<br>Example: `type=121` and `number=1` would be `y1` |
| position | Position of fragment ion from N-terminus (zero indexed).<br>Example: `type=98` and `position=2` would be `b3` |

In [24]:
row = spectral_library_flat.fragment_df.iloc[0]
row

mz           801.486877
intensity      0.125327
type         121.000000
loss_type      0.000000
charge         1.000000
number         6.000000
position       0.000000
Name: 0, dtype: float64

#### 2.2 Precursor data

The identified precursors following search are stored in the `precursor_df` DataFrame.
The precursors in this dataframe come from the spectral library but have aditional information on their identification.

The most important columns are:
- `mod_seq_charge_hash`: the hash of the precursor sequence and charge

The scans where they were identified called frames.
- `frame_start`: the frame number of the first frame in the run.
- `frame_stop`: the frame number of the last frame in the run

Furthermore there is the q-value and a multitude of scores that were used to identify the precursor.

In [26]:
precursor_df.head()

,base_width_mobility,base_width_rt,rt_observed,mobility_observed,mono_ms1_intensity,top_ms1_intensity,sum_ms1_intensity,weighted_ms1_intensity,weighted_mass_deviation,weighted_mass_error,...,_candidate_idx,valid,candidate_idx,run,mod_seq_hash,mod_seq_charge_hash,pg_master,pg,pg_qval,intensity
0,0.0,23.255554,685.877808,0.000001,1.020231e+10,1.020231e+10,1.767223e+10,7.734530e+09,-0.178309,0.178309,...,28615,True,28615,20231023_OA3_TiHe_ADIAMA_HeLa_200ng_Evo01_21mi...,14751110910839796002,14751110910839796004,P62805,P62805,0.0,4.446850e+09
1,0.0,20.185303,878.493469,0.000001,7.403505e+09,7.403505e+09,1.508568e+10,5.548682e+09,0.023692,0.023692,...,250551,True,250551,20231023_OA3_TiHe_ADIAMA_HeLa_200ng_Evo01_21mi...,13886784644007243628,13886784644007243630,P62805,P62805,0.0,4.446850e+09
2,0.0,35.747009,800.835754,0.000001,3.724941e+09,3.724941e+09,7.883017e+09,2.872972e+09,-0.070528,0.070528,...,250964,True,250964,20231023_OA3_TiHe_ADIAMA_HeLa_200ng_Evo01_21mi...,12617847998761240506,12617847998761240508,P06733,P06733,0.0,5.329763e+08
3,0.0,17.024841,489.280182,0.000001,5.295387e+09,5.295387e+09,9.861744e+09,4.008995e+09,0.333453,0.333453,...,94164,True,94164,20231023_OA3_TiHe_ADIAMA_HeLa_200ng_Evo01_21mi...,12739187365774023873,12739187365774023875,P62805,P62805,0.0,4.446850e+09
4,0.0,38.782288,908.019531,0.000001,5.116611e+07,9.041650e+07,2.270432e+08,6.594732e+07,0.386323,0.386323,...,489471,True,489471,20231023_OA3_TiHe_ADIAMA_HeLa_200ng_Evo01_21mi...,8590257174128188000,8590257174128188003,Q07021,Q07021,0.0,1.411986e+08


#### 2.3 DIA data

Last, we have the raw DIA data objecrt loaded from the Thermo raw file.

This object contains all scans in the `dia_data.spectrum_df` DataFrame.
Each spectrum points to a collection of peak based on the `peak_start_idx` and `peak_stop_idx` columns.

These point to the `dia_data.peak_df` DataFrame, which contains the peak information.

In [28]:
dia_data.spectrum_df.head()

,spec_idx,peak_start_idx,peak_stop_idx,rt,precursor_mz,precursor_charge,isolation_lower_mz,isolation_upper_mz,ms_level,nce
0,0,0,35,0.000000,-1.00000,0,-1.000000,-1.000000,1,0.0
1,1,35,42,0.000439,385.42508,0,384.424625,386.425535,2,25.0
2,2,42,43,0.000521,389.42691,0,388.426455,390.427365,2,25.0
3,3,43,44,0.000601,395.42963,0,394.429175,396.430085,2,25.0
4,4,44,45,0.000694,397.43054,0,396.430085,398.430995,2,25.0


In [29]:
dia_data.peak_df.head()

,mz,intensity
0,459.638519,13105.375977
1,470.064972,10583.207031
2,472.338043,10462.697266
3,479.018433,10998.292969
4,507.042847,13371.034180


#### 3 Map precursor hit from search results to raw data

Using this information we can map the identified precursors to the raw data.

We will use the `get_library_entry_by_hash` function to get the library entry for a given hash.

This function returns the library entry, the fragment m/z values and the fragment intensities.



In [43]:
type_map = {97:'a', 98:'b', 99: 'c', 120: 'x', 121:'y', 12:'z'}
loss_map = {0: '', 18 : 'H2O', 17: 'NH3'}
def get_ion_labels(row):
    ls_label = loss_map[int(row["loss_type"])]
    ls_label = '_'+ls_label if ls_label!='' else ''
    label = f'{type_map[row["type"]]}{int(row["number"])}{ls_label}(+{int(row["charge"])})'
    return label
spectral_library_flat.fragment_df['label'] = spectral_library_flat.fragment_df.apply(get_ion_labels, axis=1)

In [56]:
hash = precursor_df['mod_seq_charge_hash'].iloc[10]

def get_library_entry_by_hash(speclib, hash, min_intensity=0.01):
    speclib_entry = speclib.precursor_df[speclib.precursor_df['mod_seq_charge_hash'] == hash].iloc[0]

    fragment_mz = speclib.fragment_mz_df.iloc[speclib_entry.frag_start_idx:speclib_entry.frag_stop_idx].to_numpy().flatten()
    fragment_intensity = speclib.fragment_intensity_df.iloc[speclib_entry.frag_start_idx:speclib_entry.frag_stop_idx].to_numpy().flatten()
    fragment_mask = fragment_intensity > min_intensity

    fragment_mz = fragment_mz[fragment_mask]
    fragment_intensity = fragment_intensity[fragment_mask]

    # sort both by mz
    fragment_order = np.argsort(fragment_mz)
    fragment_mz = fragment_mz[fragment_order]
    fragment_intensity = fragment_intensity[fragment_order]

def get_flat_library_entry_by_hash(speclib_flat, hash, min_intensity=0.01):
    speclib_entry = speclib_flat.precursor_df[speclib_flat.precursor_df['mod_seq_charge_hash'] == hash].iloc[0]

    flat_frag_start_idx = speclib_entry.flat_frag_start_idx
    flat_frag_stop_idx = speclib_entry.flat_frag_stop_idx

    fragment_mz = speclib_flat.fragment_df['mz'].iloc[flat_frag_start_idx:flat_frag_stop_idx].to_numpy().flatten()
    fragment_intensity = speclib_flat.fragment_df['intensity'].iloc[flat_frag_start_idx:flat_frag_stop_idx].to_numpy().flatten()
    fragment_label = speclib_flat.fragment_df['label'].iloc[flat_frag_start_idx:flat_frag_stop_idx].to_numpy().flatten()
    print('fragment_label', fragment_label[:5])
    fragment_mask = fragment_intensity > min_intensity

    fragment_mz = fragment_mz[fragment_mask]
    fragment_intensity = fragment_intensity[fragment_mask]
    fragment_label = fragment_label[fragment_mask]

    # sort both by mz
    fragment_order = np.argsort(fragment_mz)
    fragment_mz = fragment_mz[fragment_order]
    fragment_intensity = fragment_intensity[fragment_order]
    fragment_label = fragment_label[fragment_order]

    return speclib_entry, fragment_mz, fragment_intensity, fragment_label

speclib_entry, mz_library, intensity_library, label_library = get_flat_library_entry_by_hash(spectral_library_flat, hash)
precursor_entry = precursor_df[precursor_df['mod_seq_charge_hash'] == hash].iloc[0]

fragment_label ['b2(+1)' 'y23(+2)' 'b3(+1)' 'y22(+2)' 'b4(+1)']


In [58]:
jit_data = dia_data.jitclass()

precursor_query = np.array([[speclib_entry.precursor_mz, speclib_entry.precursor_mz]], dtype=np.float32)
scan_limits = np.array([[precursor_entry.scan_start, precursor_entry.scan_stop, 1]], dtype=np.int64)
frame_limits = np.array([[precursor_entry.frame_start, precursor_entry.frame_stop, 1]], dtype=np.int64)

dense, precursor_index = jit_data.get_dense(
    frame_limits,
    scan_limits,
    mz_library,
    30,
    precursor_query,
)

#### 4 Visualize precursor data

Now, we want to viosualize the retrieved spectrum data.
We will start by visualizing the observed spectrum and the library spectrum.

The spectrum data `dense` is a 5 dimensional numpy array with a dense slice of the spectrum space.
The dimensions are:
- 0: either intensity information 0 or relative mass error 1
- 1: index of the fragment mz which was queried
- 2: ion mobility dimension (will be zero for DIA data)
- 3: The observations in the DIA cycle. As there might be multiple quadrupole windows where the precursor was detected, this will be a list of observations.
- 4: Retention time datapoints.

First we will select the intensity dimension and sum over all other dimensions but the fragment mz dimension.


In [61]:
from mirror_plotting import plot_mirror_byRT, split_dense_byRT

In [63]:
#df_obs = split_dense_byRT(dense, mz_library, label_library)
#df_obs

In [184]:
plot_mirror_byRT(dense, mz_library, intensity_library, label_library, width=700, height=300)

7


alt.LayerChart(...)

In [176]:
#from mirror_plotting import get_palette_for_RT
#sns.color_palette(get_palette_for_RT(7))

[(0.0, 1.0, 0.0),
 (0.7176470588235294, 0.2784313725490196, 0.0),
 (0.8823529411764706, 0.11372549019607843, 0.0),
 (1.0, 0.0, 0.0),
 (1.0, 0.6588235294117647, 0.0),
 (1.0, 0.8117647058823529, 0.0),
 (1.0, 0.9137254901960784, 0.0)]

Finally, we will visualize the Precusor ion chromatogram.
We will again select the intensity dimension and sum over ion mobility and observation but leave the retention time dimension.


In [48]:
xic_observed = dense[0].sum(axis=(1,2))
for i in range(xic_observed.shape[0]):
    plt.plot(xic_observed[i])
plt.show()